<a href="https://colab.research.google.com/github/Belac44/Python-Projects/blob/main/KMpas_OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
class StateDiagram:
  def __init__(self, numbers, bits):
    self.numbers = numbers
    self.bits = bits
    self.present_state_values = {}
    self.next_state_values = {}
    self.Qs = [(0, 0), (0, 1), (1, 0), (1, 1)]
    self.JK = [(0, 'x'), (1, 'x'), ('x', 1), ('x', 0)]

  def get_n_present_and_next_keys(self):
    """
      Calculate how many bits you will need and and anottate them as so

            Present State
        Qd    Qc     Qb     Qb
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'

    present = []
    next = []

    for i in range(self.bits-1, -1, -1):
      present_value = "Q("+ letters[i] + ")"
      next_value = "Q("+ letters[i] + "+1)"
      present.append(present_value)
      next.append(next_value)
    return [present, next]

  def make_keys_of_present_and_next_state_values(self, keys):
    """
      Use the keys generated to make a skeleton of the self.present_state_values dict's keys
      as so

      present_state_keys  = {
        "Q(d)": [],
        "Q(b)": [],
        "Q(c)":[],
        "Q(a)":[]
      }
    """
    for key in keys[0]:
      self.present_state_values[key] = []

    for key in keys[1]:
      self.next_state_values[key] = []



  def fill_present_state_keys(self):
    """
      Fill the present_state_values with there binary count of the self.numbers
    """
    for i, number in enumerate(self.numbers):
      width = len(self.present_state_values)
      binary = np.binary_repr(number, width=width)

      for i, (key, _) in enumerate(self.present_state_values.items()):
        self.present_state_values[key].append(int(binary[i]))

  def fill_next_state_keys(self):
    """
      Change the format of numbers and represent it in binary
    """
    start = self.numbers[0]
    numbers = self.numbers[1::]
    numbers.append(start)

    for i, number in enumerate(numbers):
      width = len(self.next_state_values)
      binary = np.binary_repr(number, width=width)

      for i, (key, _) in enumerate(self.next_state_values.items()):
        self.next_state_values[key].append(int(binary[i]))

  def make_dataframe(self):
    # Concatenate the two state disgrams to make a single table

    present_state = pd.DataFrame(self.present_state_values,index=self.numbers)
    next_state = pd.DataFrame(self.next_state_values, index=self.numbers)
    df = pd.concat([present_state, next_state], axis=1, keys=["Present State", "Next State"])
    return df

  def get_state_variables(self):
    #Get state variables i.e Je, Ke, Jd, Kd
    states = []
    states_dict = {}
    for key, _ in self.present_state_values.items():
      letter = key[2]
      state1 = 'J' + letter
      states.append(state1)
      state2 = 'K' + letter
      states.append(state2)

    for state in states:
      states_dict[state] = []

    return (states, states_dict)


  def fill_state_values(self, df, states):
    #Compare Present state value and Next state value to get J and K
    [state, states_dict] = states
    for i in range(len(self.numbers)):
      p = df["Present State"].iloc[i].values
      n = df["Next State"].iloc[i].values

      for j in range(self.bits):
        tup = (p[j], n[j])
        result = self.Qs.index(tup)
        value = self.JK[result]
        key = state[j * 2]
        states_dict[key].append(value[0])
        key = state[j * 2 + 1]
        states_dict[key].append(value[1])
    return states_dict


  def join_all_data(self, df, states_dict):
    for key, _ in states_dict.items():
      df[key] = states_dict[key]
    return df


  def get_table(self):
    keys = self.get_n_present_and_next_keys()
    self.make_keys_of_present_and_next_state_values(keys)
    self.fill_present_state_keys()
    self.fill_next_state_keys()
    df = self.make_dataframe()
    states = self.get_state_variables()
    states_dict = self.fill_state_values(df, states)
    table = self.join_all_data(df, states_dict)
    return table

In [ ]:
state = StateDiagram(numbers = [2,3,5,7], bits = 3)
table = state.get_table()
table

Present State           Next State               Jc Kc Jb Kb Ja Ka
           Q(c) Q(b) Q(a)     Q(c+1) Q(b+1) Q(a+1)                  
2             0    1    0          0      1      1  0  x  x  0  1  x
3             0    1    1          1      0      1  1  x  x  1  x  0
5             1    0    1          1      1      1  x  0  1  x  x  0
7             1    1    1          0      1      0  x  1  x  0  x  1